In [1]:
getwd()

[1] "/hpc/hub_oudenaarden/agiladi/data/tuft_release/release_code"

In [2]:
setwd("..")

In [3]:
library(devtools)
#source_url("https://raw.githubusercontent.com/aygoldberg/PIC-seq/master/scripts/metacell_functions.r")
source("release_code/metacell_functions.r")
library("Seurat")
library(gtools)
library(metacell)
library(GEOquery)

Loading required package: reshape2
Warning message:
“package ‘reshape2’ was built under R version 3.6.3”Loading required package: plyr
Loading required package: plotrix

Attaching package: ‘metacell’

The following objects are masked _by_ ‘.GlobalEnv’:

    mc_colorize_sup_hierarchy, mcell_mat_rpt_cor_anchors,
    mcell_mc_export_tab

Loading required package: Biobase
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
  

In [4]:
gse_id = "GSE233451"
gse=getGEO(gse_id)
filePaths = getGEOSuppFiles(gse_id)

Found 2 file(s)
GSE233451-GPL18573_series_matrix.txt.gz

── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM7428256 = col_character(),
  GSM7428257 = col_character(),
  GSM7428258 = col_character(),
  GSM7428259 = col_character(),
  GSM7428260 = col_character(),
  GSM7428261 = col_character(),
  GSM8378150 = col_character(),
  GSM8378151 = col_character()
)
File stored at: 
/scratch/22337103/RtmpQnEK94/GPL18573.soft
GSE233451-GPL24676_series_matrix.txt.gz

── Column specification ────────────────────────────────────────────────────────
cols(
  ID_REF = col_character(),
  GSM7428262 = col_character()
)
File stored at: 
/scratch/22337103/RtmpQnEK94/GPL24676.soft


In [5]:
filePaths

,size,isdir,mode,mtime,ctime,atime,uid,gid,uname,grname
/hpc/hub_oudenaarden/agiladi/data/tuft_release/GSE233451/GSE233451_RAW.tar,290273280,FALSE,644,2024-07-14 19:33:07,2024-07-14 19:33:07,2024-07-14 19:33:07,32419,10308,agiladi,hub_oudenaarden
/hpc/hub_oudenaarden/agiladi/data/tuft_release/GSE233451/GSE233451_metadata.txt.gz,57746,FALSE,644,2024-07-14 19:33:08,2024-07-14 19:33:08,2024-07-14 19:33:08,32419,10308,agiladi,hub_oudenaarden


In [6]:
raw_path = grep("RAW", rownames(filePaths), v=T)
umidir = "release_umi.tab/"
dir.create(umidir)
untar(raw_path, exdir = umidir)

Warning message in dir.create(umidir):
“'release_umi.tab' already exists”

In [7]:
setwd(umidir)
#system("gzip -d *.gz")
system("ls -1 | awk -F'_' '{print $0,$2}' | xargs -n 2 mv")
setwd("..")

In [8]:
metadata_fn = grep("metadata", rownames(filePaths), v=T)
cell_metadata = read.delim(metadata_fn, stringsAsFactors = F, row.names = 1)
head(cell_metadata)
table(cell_metadata$amp_batch_id)

,amp_batch_id,cell_barcode,umicount,mt_count,coord,source,type,gating,treatment,age,clover
HUB.JO.s037_AAACAGGC,HUB.JO.s037,AAACAGGC,7296,1280,H18,Ileum,organoid,AVIL-Clover+,no_cyto,adult,AVIL
HUB.JO.s037_AAAGCGGA,HUB.JO.s037,AAAGCGGA,3461,626,P12,Ileum,organoid,AVIL-Clover+,IL4+IL13,adult,AVIL
HUB.JO.s037_AAAGGCTG,HUB.JO.s037,AAAGGCTG,4062,1019,O10,Ileum,organoid,AVIL-Clover+,IL4+IL13,adult,AVIL
HUB.JO.s037_AACACGCA,HUB.JO.s037,AACACGCA,8352,1471,L20,Ileum,organoid,AVIL-Clover+,IL4+IL13,adult,AVIL
HUB.JO.s037_AACATGGG,HUB.JO.s037,AACATGGG,4986,692,P10,Ileum,organoid,AVIL-Clover+,IL4+IL13,adult,AVIL
HUB.JO.s037_AACCCAAC,HUB.JO.s037,AACCCAAC,8453,1449,O15,Ileum,organoid,AVIL-Clover+,IL4+IL13,adult,AVIL



HUB.JO.s037 HUB.JO.s042 HUB.JO.s043 HUB.JO.s054 HUB.JO.s057 HUB.JO.s058 
        384         384         380         383         384         384 
HUB.LH.s001 HUB.LH.s002 HUB.LH.s003 HUB.LH.s009 
        383         384         384         384 

In [9]:
umi_files = list.files(umidir, pattern = "*.txt")
data_list = lapply(paste0(umidir, "/", umi_files), read.delim, stringsAsFactor=F, row.names=1)
length(data_list)
sapply(data_list, nrow)

[1] 8

[1] 25267 25502 24643 26898 25126 22283 25742 24955

In [ ]:
names(data_list) = vecsplit(umi_files, "\\.", 1)
names(data_list)

In [ ]:
genes = names(table(unlist(lapply(data_list, rownames))))
length(genes)
tail(genes)

In [ ]:
cells = #paste0(rep(names(data_list), sapply(data_list, ncol)), "_", 
               sapply(data_list, colnames)
length(cells)
sapply(cells, length)

In [ ]:
cells = #paste0(rep(names(data_list), sapply(data_list, ncol)), "_", 
               unlist(sapply(data_list, colnames))
head(cells)
tail(cells)

In [ ]:
data = matrix(0, nrow = length(genes), ncol = length(cells), dimnames = list(genes, cells))
for (samp in names(data_list)) {
    X = data_list[[samp]]
    shared = intersect(genes, rownames(X))
    message(samp, ": ", length(shared), " genes and ", ncol(X), " cells")    
    #data[ shared, paste0(samp, "_", colnames(X))] = as.matrix(X[shared,])
    data[ shared, colnames(X)] = as.matrix(X[shared,])
}

In [ ]:
venn(colnames(data), rownames(cell_metadata))

In [ ]:
dir.create("saved_work/")
scdb_init("saved_work/", force_reinit=T)
id = "tuft"
id_all = "tuft_all"

In [ ]:
scdb_add_mat(id_all, tgScMat(mat=data, cell_metadata=cell_metadata[ colnames(data),]))

In [ ]:
bad_genes = unique(c(grep("ERCC|^MT-", rownames(data), v=T)))
bad_genes

In [ ]:
import_metacell_structure(id = "ileum_organoids", 
                        folder = "release_import/figure3_cs2/", 
                        all_id = "tuft_all", 
                        bad_genes = bad_genes, 
                        url = NULL)

## Import 10x data

In [ ]:
list.files(umidir)

In [ ]:
umidir = "release_umi.tab/"

In [ ]:
data = Seurat::Read10X(data.dir = umidir)

In [ ]:
cellplex = as.matrix(data[[2]])
dim(cellplex)
head(t(cellplex))

In [ ]:
first = apply(cellplex, 2, max)
second = apply(cellplex, 2, function(x) sort(x,T)[2])

roof = 2e5
idx = sample(seq_along(first), roof)
x = log2(second + 1)
y = log2(first + 1)
z = log2((first + 1) / (second + 1))
plot(x[idx],y[idx],pch = 20, cex = 0.7)
grid(col = "blue")
abline(coef = c(0,1))
abline(coef = c(3,1), lty=2, lwd=2)
abline(h=7, lty=2, lwd=2)
#table(first > 0, second > 0)

In [ ]:
y = log2(first + 1)
z = log2((first + 1) / (second + 1))

good_cells = names(which(y > 6 & z > 3))
length(good_cells)
head(good_cells)

In [ ]:
data = as.matrix(data[[1]][,good_cells])
dim(data)

In [ ]:
tag = rownames(cellplex)[ max.col(t(cellplex))]
names(tag) = colnames(cellplex)
head(tag)
table(tag[ good_cells])

In [ ]:
cellplex_md = read.delim("release_import/figure4_10x/tuft_10x_metadata.txt", stringsAsFactors = F, row.names = 1)
cellplex_md

In [ ]:
cell_metadata = cbind(tag = tag[ good_cells], cellplex_md[ tag[ good_cells],], umicount = colSums(data[,good_cells]))
rownames(cell_metadata) = good_cells
head(cell_metadata)
tail(cell_metadata)

In [ ]:
colnames(data) = paste0(colnames(data), "-1")
rownames(cell_metadata) = paste0(rownames(cell_metadata), "-1")
head(colnames(data))
head(cell_metadata)

In [ ]:
id = "10x"
id_all = "10x_all"

In [ ]:
scdb_add_mat(id_all, tgScMat(mat=data[, rownames(cell_metadata)], cell_metadata=cell_metadata))

In [ ]:
bad_genes = unique(c(grep("^MT-|^ERCC|^IG[HLK]", rownames(data), v=T)))
bad_genes

In [ ]:
import_metacell_structure(id = id, 
                        folder = "release_import/figure4_10x/", 
                        all_id = id_all, 
                        bad_genes = bad_genes, 
                        url = NULL)

## Download and organize published data

In [ ]:
# Teichman's data

setwd("release_umi.tab/")
system("wget https://cellgeni.cog.sanger.ac.uk/gutcellatlas/Full_obj_raw_counts_nosoupx_v2.h5ad")
setwd("..")

In [ ]:
library(SeuratDisk)

Convert("release_umi.tab/Full_obj_raw_counts_nosoupx_v2.h5ad", dest="h5seurat", overwrite=T)
obj = LoadH5Seurat("release_umi.tab/Full_obj_raw_counts_nosoupx_v2.h5seurat")


In [ ]:
# Hickey's data

links = read.delim("release_import/figure1_Hickey/links.txt", stringsAsFactors = F, h=F, skip = 1)
data_files = grep("\\.gz", links[,"V2"])
links$folder = "./"
links[ data_files, "folder"] = vecsplit(links[ data_files, "V2"], "_", 1)
links$file = links$V2
links[ data_files, "file"] = vecsplit(links[ data_files, "V2"], "_", 2)

#links$file = vecsplit(links$V2, "_", 2)
links

curdir = getwd()
umidir = paste0(curdir, "/release_umi.tab/figure1_Hickey")
umidir
dir.create(umidir)

head(links, 10)

setwd(umidir)

p = function(i, links) {
    message(links[i, "V2"])
    dir.create(links[i, "folder"], showWarnings = F)
    cmd1 = paste0(curdir, "/release_code/download_wrapper.sh ", links[i, "V3"], " ", links[i, "V1"], " ", links[i, "folder"], "/", links[i, "file"])
    message(cmd1)
    system(cmd1)
    
}

#invisible(sapply(seq_len(nrow(links)), p, links))
invisible(sapply(seq_len(1), p, links))

setwd(curdir)

setwd(umidir)
folders = names(table(links$folder))
invisible(sapply(folders, dir.create, showWarnings = F))
setwd(curdir)

cmds = sapply(seq_len(nrow(links)), function(i) 
    paste0(curdir, "/release_code/download_wrapper.sh ", links[i, "V3"], " ", links[i, "V1"], " ", links[i, "folder"], "/", links[i, "file"]))
write.table(cmds, quote = F, row.names = F)

setwd(umidir)
i = 1
    message(links[i, "V2"])
    dir.create(links[i, "folder"], showWarnings = F)
    setwd(links[i, "folder"])
    message(getwd())
    #cmd1 = paste0('wget --user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36" ',
    #             links[i, "V3"])

    cmd1 = paste0(curdir, "/release_code/download_wrapper.sh ", links[i, "V3"], " ", links[i, "V1"], " ", links[i, "file"])
    message(cmd1)
    system(cmd1)
    #cmd2 = paste0("mv ", links[i, "V1"], " ", links[i, "file"])
    #message(cmd2)
    #system(cmd2)
setwd(curdir)


#setwd("/hpc/hub_oudenaarden/agiladi/data/tuft_release/")
getwd()


system('wget http://datadryad.org/stash/downloads/file_stream/2121068')


In [4]:
umidir = "release_umi.tab/figure1_Hickey/"
files = list.files(umidir)
files

[1] "B001-A-001"                             
 [2] "B001-A-006"                             
 [3] "B001-A-101"                             
 [4] "B001-A-201"                             
 [5] "B001-A-301"                             
 [6] "B001-A-401"                             
 [7] "B001-A-406"                             
 [8] "B001-A-501"                             
 [9] "B004-A-004"                             
[10] "B004-A-008"                             
[11] "B004-A-104"                             
[12] "B004-A-104-R2"                          
[13] "B004-A-204"                             
[14] "B004-A-304"                             
[15] "B004-A-404"                             
[16] "B004-A-404-R2"                          
[17] "B004-A-408"                             
[18] "B004-A-408-R2"                          
[19] "B004-A-504"                             
[20] "B004-A-504-R2"                          
[21] "B005-A-001"                             
[22] "B005-A-002"                             
[23] "B005-A-101"                             
[24] "B005-A-201"                             
[25] "B005-A-301"                             
[26] "B005-A-401"                             
[27] "B005-A-402"                             
[28] "B005-A-501"                             
[29] "B005-A-501-R2"                          
[30] "B006-A-001"                             
[31] "B006-A-002"                             
[32] "B006-A-101"                             
[33] "B006-A-201"                             
[34] "B006-A-201-R2"                          
[35] "B006-A-301"                             
[36] "B006-A-401"                             
[37] "B006-A-402"                             
[38] "B006-A-501"                             
[39] "B008-A-001"                             
[40] "B008-A-002"                             
[41] "B008-A-101"                             
[42] "B008-A-201"                             
[43] "B008-A-301"                             
[44] "B008-A-401"                             
[45] "B008-A-402"                             
[46] "B008-A-501"                             
[47] "B009-A-001"                             
[48] "B009-A-101"                             
[49] "B009-A-301"                             
[50] "B009-A-401"                             
[51] "B009-A-405"                             
[52] "B010-A-001"                             
[53] "B010-A-002"                             
[54] "B010-A-101"                             
[55] "B010-A-201"                             
[56] "B010-A-301"                             
[57] "B010-A-401"                             
[58] "B010-A-405"                             
[59] "B010-A-501"                             
[60] "B011-A-001"                             
[61] "B011-A-002"                             
[62] "B011-A-101"                             
[63] "B011-A-201"                             
[64] "B011-A-301"                             
[65] "B011-A-401"                             
[66] "B011-A-405"                             
[67] "B011-A-501"                             
[68] "B012-A-001"                             
[69] "B012-A-002"                             
[70] "B012-A-101"                             
[71] "B012-A-201"                             
[72] "B012-A-301"                             
[73] "B012-A-401"                             
[74] "B012-A-405"                             
[75] "B012-A-501"                             
[76] "clustered_colon_object.rds"             
[77] "clustered_duodenum_object.rds"          
[78] "clustered_enteroendocrine_object.rds"   
[79] "clustered_ileum_object.rds"             
[80] "clustered_immune_object.rds"            
[81] "clustered_jejunum_object.rds"           
[82] "clustered_secretory_special_object.rds" 
[83] "clustered_stromal_object.rds"           
[84] "Colon_Epithelial_UMAP_CellType.tsv"     
[85] "datasets_table_fn.txt"                  
[86] "

In [5]:
metadata_fn = grep("Epithelial", files, v=T)
metadata_fn
metadata_list = lapply(paste0(umidir, "/", metadata_fn), read.delim, stringsAsFactors = F, row.names = 1)
length(metadata_list)
head(metadata_list[[1]])

[1] "Colon_Epithelial_UMAP_CellType.tsv"   
[2] "Duodenum_Epithelial_UMAP_CellType.tsv"
[3] "Ileum_Epithelial_UMAP_CellType.tsv"   
[4] "Jejunum_Epithelial_UMAP_CellType.tsv"

[1] 4

,UMAP_1,UMAP_2,CellType
B001-A-301_AAACCCACAGCTATTG-1,2.292168,-3.326214,TA1
B001-A-301_AAACCCAGTGTTCCAA-1,4.247404,6.580899,Enterocytes
B001-A-301_AAACGAAGTGGTATGG-1,4.615074,5.245791,Immature Enterocytes
B001-A-301_AAACGAATCCAATGCA-1,4.477911,-0.670288,TA1
B001-A-301_AAACGAATCGTGGCGT-1,0.353033,-3.104455,TA1
B001-A-301_AAACGCTAGACGATAT-1,5.073505,6.059951,Enterocytes


In [6]:
l = sapply(metadata_list, nrow)
l

[1] 72269 39292 23627 65197

In [7]:
metadata = do.call(rbind, metadata_list)
nrow(metadata)

[1] 200385

In [8]:
metadata$tissue = rep(vecsplit(metadata_fn, "_", 1), l)
metadata$folder = vecsplit(rownames(metadata), "_", 1)
head(metadata)

,UMAP_1,UMAP_2,CellType,tissue,folder
B001-A-301_AAACCCACAGCTATTG-1,2.292168,-3.326214,TA1,Colon,B001-A-301
B001-A-301_AAACCCAGTGTTCCAA-1,4.247404,6.580899,Enterocytes,Colon,B001-A-301
B001-A-301_AAACGAAGTGGTATGG-1,4.615074,5.245791,Immature Enterocytes,Colon,B001-A-301
B001-A-301_AAACGAATCCAATGCA-1,4.477911,-0.670288,TA1,Colon,B001-A-301
B001-A-301_AAACGAATCGTGGCGT-1,0.353033,-3.104455,TA1,Colon,B001-A-301
B001-A-301_AAACGCTAGACGATAT-1,5.073505,6.059951,Enterocytes,Colon,B001-A-301


In [9]:
tenxruns = names(table(metadata$folder))
tenxruns

[1] "B001-A-001"    "B001-A-006"    "B001-A-101"    "B001-A-201"   
 [5] "B001-A-301"    "B001-A-401"    "B001-A-406"    "B001-A-501"   
 [9] "B004-A-004"    "B004-A-008"    "B004-A-104"    "B004-A-104-R2"
[13] "B004-A-204"    "B004-A-304"    "B004-A-404"    "B004-A-404-R2"
[17] "B004-A-408"    "B004-A-408-R2" "B004-A-504"    "B004-A-504-R2"
[21] "B005-A-001"    "B005-A-002"    "B005-A-101"    "B005-A-201"   
[25] "B005-A-301"    "B005-A-401"    "B005-A-402"    "B005-A-501"   
[29] "B005-A-501-R2" "B006-A-001"    "B006-A-002"    "B006-A-101"   
[33] "B006-A-201"    "B006-A-201-R2" "B006-A-301"    "B006-A-401"   
[37] "B006-A-402"    "B006-A-501"    "B008-A-001"    "B008-A-002"   
[41] "B008-A-101"    "B008-A-201"    "B008-A-301"    "B008-A-401"   
[45] "B008-A-402"    "B008-A-501"    "B009-A-001"    "B009-A-101"   
[49] "B009-A-301"    "B009-A-401"    "B009-A-405"    "B010-A-001"   
[53] "B010-A-002"    "B010-A-101"    "B010-A-201"    "B010-A-301"   
[57] "B010-A-401"    "B010-A-405"    "B010-A-501"    "B011-A-001"   
[61] "B011-A-002"    "B011-A-101"    "B011-A-201"    "B011-A-301"   
[65] "B011-A-401"    "B011-A-405"    "B011-A-501"    "B012-A-001"   
[69] "B012-A-002"    "B012-A-101"    "B012-A-201"    "B012-A-301"   
[73] "B012-A-401"    "B012-A-405"    "B012-A-501"

In [10]:
tenxfiles = t(sapply(tenxruns, function(x) setdiff(list.files(paste0(umidir, "/", x)), "")))
tenxfiles

B001-A-001,barcodes.tsv.gz,features.tsv.gz,matrix.mtx.gz
B001-A-006,barcodes.tsv.gz,features.tsv.gz,matrix.mtx.gz
B001-A-101,barcodes.tsv.gz,features.tsv.gz,matrix.mtx.gz
B001-A-201,barcodes.tsv.gz,features.tsv.gz,matrix.mtx.gz
B001-A-301,barcodes.tsv.gz,features.tsv.gz,matrix.mtx.gz
B001-A-401,barcodes.tsv.gz,features.tsv.gz,matrix.mtx.gz
B001-A-406,barcodes.tsv.gz,features.tsv.gz,matrix.mtx.gz
B001-A-501,barcodes.tsv.gz,features.tsv.gz,matrix.mtx.gz
B004-A-004,barcodes.tsv.gz,features.tsv.gz,matrix.mtx.gz
B004-A-008,barcodes.tsv.gz,features.tsv.gz,matrix.mtx.gz
B004-A-104,barcodes.tsv.gz,features.tsv.gz,matrix.mtx.gz


In [11]:
files_tab = cbind(Batch.Set.ID = tenxruns, 
                  mat_fn = paste0(tenxruns, "/", tenxfiles[,3]), 
                  genes_fn = paste0(tenxruns, "/", tenxfiles[,2]),
                  cells_fn = paste0(tenxruns, "/", tenxfiles[,1]))
files_tab

write.table(files_tab, quote=F, sep = "\t", row.names = F, file = paste0(umidir, "/datasets_table_fn.txt"))

Batch.Set.ID,mat_fn,genes_fn,cells_fn
B001-A-001,B001-A-001/matrix.mtx.gz,B001-A-001/features.tsv.gz,B001-A-001/barcodes.tsv.gz
B001-A-006,B001-A-006/matrix.mtx.gz,B001-A-006/features.tsv.gz,B001-A-006/barcodes.tsv.gz
B001-A-101,B001-A-101/matrix.mtx.gz,B001-A-101/features.tsv.gz,B001-A-101/barcodes.tsv.gz
B001-A-201,B001-A-201/matrix.mtx.gz,B001-A-201/features.tsv.gz,B001-A-201/barcodes.tsv.gz
B001-A-301,B001-A-301/matrix.mtx.gz,B001-A-301/features.tsv.gz,B001-A-301/barcodes.tsv.gz
B001-A-401,B001-A-401/matrix.mtx.gz,B001-A-401/features.tsv.gz,B001-A-401/barcodes.tsv.gz
B001-A-406,B001-A-406/matrix.mtx.gz,B001-A-406/features.tsv.gz,B001-A-406/barcodes.tsv.gz
B001-A-501,B001-A-501/matrix.mtx.gz,B001-A-501/features.tsv.gz,B001-A-501/barcodes.tsv.gz
B004-A-004,B004-A-004/matrix.mtx.gz,B004-A-004/features.tsv.gz,B004-A-004/barcodes.tsv.gz
B004-A-008,B004-A-008/matrix.mtx.gz,B004-A-008/features.tsv.gz,B004-A-008/barcodes.tsv.gz


In [15]:
table(metadata$CellType, metadata$tissue)
tuft_cells = rownames(metadata)[ metadata$CellType == "Tuft" & metadata$tissue == "Jejunum"]
length(tuft_cells)
head(tuft_cells)

                             
                              Colon Duodenum Ileum Jejunum
  Best4+ Enterocytes           4028      603   274    1343
  CyclingTA 1                  9204     1609   622    4278
  CyclingTA 2                   886        0     0       0
  Enterocytes                  7655     5055  3475    5312
  Enteroendocrine               628      802   722    1501
  Epithelial                      0     4434  3850    4704
  Goblet                       2312      861   576    1803
  Immature Enterocytes         8955    15234  7791   22889
  Immature Goblet              8311      405  1432    1876
  Paneth                          0      289   251    1938
  Secretory Specialized MUC6+     0     1740     0       0
  Stem                         3351      694   914    2127
  TA1                         15884     2895  2138    9988
  TA2                         10834     4474  1544    6748
  Tuft                          221      170    38     690
  Unknown                 

[1] 690

[1] "B001-A-001_AAAGGGCAGCTGGCCT-1" "B001-A-001_AGGAATACAAAGCTAA-1"
[3] "B001-A-001_AGGTCATGTGACAGCA-1" "B001-A-001_CAAGGGAGTAGACGGT-1"
[5] "B001-A-001_CAGGTATGTTACGGAG-1" "B001-A-001_CCACCATAGGATAATC-1"

In [20]:
rel_folders = names(table(metadata[ tuft_cells, "folder"]))
rel_folders
length(rel_folders)

[1] "B001-A-001"    "B001-A-101"    "B004-A-404"    "B004-A-404-R2"
 [5] "B004-A-408"    "B004-A-408-R2" "B005-A-401"    "B005-A-402"   
 [9] "B006-A-401"    "B006-A-402"    "B008-A-401"    "B008-A-402"   
[13] "B009-A-401"    "B009-A-405"    "B010-A-401"    "B010-A-405"   
[17] "B011-A-401"    "B011-A-405"    "B012-A-401"    "B012-A-405"

[1] 20

data_merged = mcell_read_multi_scmat_10x(datasets_table_fn = paste0(umidir, "/datasets_table_fn.txt"), base_dir = umidir)

In [21]:
data_list = lapply(rel_folders, function(f) {
    data = Seurat::Read10X(data.dir = paste0(umidir, "/", f))
    if (length(data) == 2) {data = data[[1]]}
    colnames(data) = paste0(f, "_", colnames(data), "-1")
    X = as.matrix(data[, intersect( tuft_cells, colnames(data))])
    message(f, ": ", nrow(X), " genes over ", ncol(X), " cells")
    X
})

B001-A-001: 33538 genes over 26 cells
B001-A-101: 33538 genes over 27 cells
B004-A-404: 33538 genes over 69 cells
10X data contains more than one type and is being returned as a list containing matrices of each type.
B004-A-404-R2: 36601 genes over 39 cells
B004-A-408: 33538 genes over 35 cells
B004-A-408-R2: 36601 genes over 17 cells
B005-A-401: 33694 genes over 31 cells
B005-A-402: 33694 genes over 45 cells
10X data contains more than one type and is being returned as a list containing matrices of each type.
B006-A-401: 36601 genes over 9 cells
10X data contains more than one type and is being returned as a list containing matrices of each type.
B006-A-402: 36601 genes over 50 cells
10X data contains more than one type and is being returned as a list containing matrices of each type.
B008-A-401: 36601 genes over 58 cells
10X data contains more than one type and is being returned as a list containing matrices of each type.
B008-A-402: 36601 genes over 93 cells
10X data contains more t

In [22]:
genes = names(table(unlist(lapply(data_list, rownames))))
length(genes)
head(genes)
tail(genes)

[1] 49632

[1] "A1BG"     "A1BG-AS1" "A1CF"     "A2M"      "A2M-AS1"  "A2ML1"

[1] "ZXDC"   "ZYG11A" "ZYG11B" "ZYX"    "ZZEF1"  "ZZZ3"

In [23]:
cells = unlist(sapply(data_list, colnames))
length(cells)

head(cells)

[1] 690

[1] "B001-A-001_AAAGGGCAGCTGGCCT-1" "B001-A-001_AGGAATACAAAGCTAA-1"
[3] "B001-A-001_AGGTCATGTGACAGCA-1" "B001-A-001_CAAGGGAGTAGACGGT-1"
[5] "B001-A-001_CAGGTATGTTACGGAG-1" "B001-A-001_CCACCATAGGATAATC-1"

In [28]:
names(data_list) = rel_folders

In [29]:
data = matrix(0, nrow = length(genes), ncol = length(cells), dimnames = list(genes, cells))
dim(data)

[1] 49632   690

In [31]:
for (samp in names(data_list)) {
    X = data_list[[samp]]
    #shared = intersect(genes, rownames(X))
    message(samp, ": ", nrow(X), " genes and ", ncol(X), " cells")    
    data[ rownames(X), colnames(X)] = as.matrix(X)
}

B001-A-001: 33538 genes and 26 cells
B001-A-101: 33538 genes and 27 cells
B004-A-404: 33538 genes and 69 cells
B004-A-404-R2: 36601 genes and 39 cells
B004-A-408: 33538 genes and 35 cells
B004-A-408-R2: 36601 genes and 17 cells
B005-A-401: 33694 genes and 31 cells
B005-A-402: 33694 genes and 45 cells
B006-A-401: 36601 genes and 9 cells
B006-A-402: 36601 genes and 50 cells
B008-A-401: 36601 genes and 58 cells
B008-A-402: 36601 genes and 93 cells
B009-A-401: 36601 genes and 24 cells
B009-A-405: 36601 genes and 33 cells
B010-A-401: 36601 genes and 10 cells
B010-A-405: 36601 genes and 26 cells
B011-A-401: 36601 genes and 10 cells
B011-A-405: 36601 genes and 11 cells
B012-A-401: 36601 genes and 37 cells
B012-A-405: 36601 genes and 40 cells


In [33]:
dim(data)
quantile(colSums(data))

[1] 49632   690

0%      25%      50%      75%     100% 
  459.00  1594.25  3172.50  6072.75 18793.00

In [ ]:
head(colnames(data))
#B001-A-301_AAACCCACAGCTATTG-1
colnames(data) = paste0(files_tab[1,1], "_", colnames(data), "-1")

X = as.matrix(data[, intersect(rownames(metadata), colnames(data))])
dim(X)


In [34]:
dir.create("saved_work/")
scdb_init("saved_work/", force_reinit=T)
id = "hickey_tuft"
id_all = "hickey_tuft"

Warning message in dir.create("saved_work/"):
“'saved_work' already exists”initializing scdb to saved_work/


In [36]:
scdb_add_mat(id_all, tgScMat(mat=data, cell_metadata=metadata[ colnames(data),]))